In [ ]:
import pandas as pd
import yfinance as yf

# Fetch the list of S&P 500 companies from Wikipedia
sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
sp500_list = sp500[0]['Symbol'].tolist()

# Fetch data in bulk for all S&P 500 companies
data = yf.download(sp500_list, period="1d", group_by='ticker')

# Extract market cap data for each ticker
market_caps = {}
for ticker in sp500_list:
    try:
        stock = yf.Ticker(ticker)
        market_caps[ticker] = stock.info['marketCap']
    except:
        print(f"Couldn't fetch data for {ticker}")
        continue

# Sort tickers by market cap
sorted_tickers = sorted(market_caps, key=market_caps.get, reverse=True)

# Extract top 100 tickers
top_100_tickers = sorted_tickers[:100]

# Print them
print(top_100_tickers)


In [1]:
import pandas as pd
import yfinance as yf

# Fetch the list of S&P 500 companies from Wikipedia
sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
sp500_list = sp500[0]['Symbol'].tolist()

# Fetch data in bulk for all S&P 500 companies
data = yf.download(sp500_list, period="1d", group_by='ticker')

# Extract market cap data for each ticker
market_caps = {}
for ticker in sp500_list:
    try:
        # Market Cap can be calculated using the formula: Market Cap = Shares Outstanding * Current Share Price
        market_caps[ticker] = data[ticker]['Close'].iloc[0] * data[ticker]['Volume'].iloc[0]
    except:
        print(f"Couldn't fetch data or calculate market cap for {ticker}")
        continue

# Sort tickers by market cap
sorted_tickers = sorted(market_caps, key=market_caps.get, reverse=True)

# Extract top 100 tickers
top_100_tickers = sorted_tickers[:100]

# Print them
print(top_100_tickers)


[*********************100%%**********************]  503 of 503 completed


2 Failed downloads:
['BF.B']: Exception("%ticker%: Period '1d' is invalid, must be one of ['1mo', '3mo', '6mo', 'ytd', '1y', '2y', '5y', '10y', 'max']")
['BRK.B']: Exception('%ticker%: No data found, symbol may be delisted')



['TSLA', 'AAPL', 'AMZN', 'GOOGL', 'AMD', 'GOOG', 'XOM', 'LLY', 'IBM', 'BAC', 'ADBE', 'HES', 'COST', 'ALGN', 'F', 'DIS', 'ANET', 'MO', 'AMT', 'HD', 'HON', 'ABBV', 'GE', 'ACN', 'ENPH', 'COP', 'AMAT', 'FDX', 'DHR', 'EW', 'GM', 'ABT', 'T', 'AMGN', 'ABNB', 'GS', 'HSY', 'ADP', 'ADI', 'DXCM', 'DG', 'AXP', 'EQIX', 'FI', 'GIS', 'AAL', 'CSX', 'FANG', 'DE', 'CVS', 'DLR', 'EMR', 'ADSK', 'GD', 'HAS', 'AZO', 'GILD', 'HUM', 'CCI', 'FCX', 'HCA', 'MMM', 'AON', 'HAL', 'ETN', 'BKR', 'APH', 'DAL', 'HLT', 'AMP', 'IDXX', 'CTRA', 'ALB', 'FTNT', 'EQT', 'ELV', 'EOG', 'DOW', 'ITW', 'D', 'AJG', 'EXC', 'EXPE', 'DUK', 'DLTR', 'FICO', 'DVN', 'GPN', 'EG', 'ETSY', 'AEP', 'DHI', 'HPQ', 'ADM', 'ALL', 'EBAY', 'AVB', 'EL', 'GEHC', 'IEX']
